In [1]:
import imageio
import numpy as np
from skimage import io, filters
from skimage.measure import regionprops, label
from skimage.color import rgb2gray
import os

def z_project_max_intensity(image):
    return np.max(image, axis=0)

def process_image(target_file, output_dir, prefix, max_divisions):
    # Load the image
    image = imageio.imread(target_file)

    # Perform Z projection (max intensity)
    projected_image = z_project_max_intensity(image)

    # Create a grayscale image if necessary
    if len(projected_image.shape) == 3:
        projected_image = rgb2gray(projected_image)

    # Iterate over threshold divisions
    for i in range(1, max_divisions + 1):
        # Set the threshold
        thresh = i
        
        # Create a binary mask
        binary_mask = projected_image > thresh
        
        # Label connected components
        labeled_mask = label(binary_mask)

        # Optionally, you can compute statistics (e.g., area, mean) for each region
        stats = regionprops(labeled_mask)

        # Save the processed image
        index = f"{i:02}"
        output_file = os.path.join(output_dir, f"{prefix}_{index}.tiff")
        io.imsave(output_file, labeled_mask.astype(np.uint8) * 255)  # Save as binary image

    print(f"Processed {max_divisions} files and saved to {output_dir}")

# Main execution
if __name__ == "__main__":
    # User input for the file and output directory
    target_file = input("Enter the path to the image file: ")
    output_dir = input("Enter the output directory for MIB masks: ")
    prefix = input("Enter the desired filename prefix: ")

    # Example for max_divisions based on the maximum pixel value
    image = imageio.imread(target_file)
    max_divisions = np.max(image)  # Determine the max threshold

    process_image(target_file, output_dir, prefix, max_divisions)


2024-09-22 13:05:35.619 Python[4636:43774] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Saved: /Users/siyangc4/Downloads/default_00.tif
Saved: /Users/siyangc4/Downloads/default_01.tif
Saved: /Users/siyangc4/Downloads/default_02.tif
Saved: /Users/siyangc4/Downloads/default_03.tif
